In [13]:
%matplotlib inline
!pip install thop

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from thop import profile
from torchsummary import summary


In [ ]:
torch.manual_seed(120)


# training a model first

In [14]:
use_cuda = True

NUM_RUNS = 3
EPOCHS = 10


In [15]:
list(range(5,0,-1))

[5, 4, 3, 2, 1]

In [16]:
def find_res_fixed_flops(N1,      C1, K1, P1, S1,     C2, K2, P2, S2, img_channels):
  '''
    Keeping FLOPS the same
    Approach III in the paper
  '''

  k_p = 2 # pooling layer kernel size


  # model_old = NetTest(im_res=N1, conv1_size=C1, conv2_size=C2, num_conv1=K1, num_conv2=K2, padd_1=P1, padd_2=P2, stride_1=S1, stride_2=S2).to(device)
  # macs_old, params_old = profile(model_old, inputs=(torch.randn(1, 1, N1, N1).to(device), ))



  Solutions = []

  # import pdb; pdb.set_trace()
  M1 = int((N1 - C1 + 2*P1)/S1 + 1) # res after first layer	
  M1_p = int((M1 - k_p + 2*0)/k_p + 1) # res after first pooling layer	
  M2  = int((M1_p  -  C2 + 2*P2)  / S2 + 1)

  for nN1 in np.arange(N1+1,29):
    for nC1 in np.arange(C1,6):#nN1//2): #3,8,2):#C1+1,nN1//2): # filter size new layer 1	
      for nP1 in range(4): #nC1//2):
        for nS1 in range(1,4):
          nM1 = int((nN1 - nC1 + 2*nP1)/nS1 + 1) # resolution after conv1
          nM1_p = int((nM1 - k_p + 2*0)/k_p + 1) # resolution after pool1 
          # if nM1 != M1: continue
                
          for nK1 in np.arange(K1-1,2,-1): # num filters new layer 1
            # if (nC1**2) * nK1 != (C1**2) * K1: continue
            
            for nC2 in np.arange(3,6,2):#nM1//2): # filter size second conv layer		 
              # if (nC2**2) * nK1 != (C2**2) * K1: continue 
        
              # 2nd layer
              for nP2 in np.arange(4): # padding new layer 2
                for nS2 in np.arange(1,4): # stride new layer 2
                  nM2 = int((nM1_p - nC2 + 2*nP2)  / nS2 + 1)

                  if nM2  != M2 : continue
                  # M3 = int( (nM1 - B1 + 2*Q2)  / R2 + 1)

                  # FLOPS_old = (K1 * M1**2 * (img_channels * C1**2 )) + (K2 * M2**2 * (K1 * C2**2 ))
                  # FLOPS_new = (nK1 * nM1**2 * (img_channels * nC1**2 )) + (K2 * M2**2 * (nK1 * nC2**2  ))



                  # discard if the oiginal model is too off from the new model in terms of the number of filters
                  if np.abs(N1 - nN1) > 


                  # import pdb; pdb.set_trace()
                  FLOPS_old = (K1 * M1**2 * (img_channels * C1**2 )) + (K2 * M2**2 * (K1 * C2**2 ))
                  FLOPS_new = (nK1 * nM1**2 * (img_channels * nC1**2 )) + (K2 * M2**2 * (nK1 * nC2**2  ))

                  # build a model and compute its MACs directly

                  # model_new = NetTest(im_res=nN1, conv1_size=nC1, conv2_size=nC2, num_conv1=nK1, num_conv2=K2, padd_1=nP1, padd_2=nP2, stride_1=nS1, stride_2=nS2).to(device)
                  # macs_new, params_new = profile(model_new, inputs=(torch.randn(1, 1,nN1, nN1).to(device), ))

                  # print(f'FLOPS old: {FLOPS_old}, FLOPS new: {FLOPS_new}\n')

                  if FLOPS_old==FLOPS_new:
                  # if macs_new==macs_old:
                    Solutions.append({'im_res_orig':N1, 'im_res_new':nN1, 'layer1_old':[C1, K1, P1, S1], 'layer2_old':[C2, K2, P2, S2], 'layer1':[nC1, nK1, nP1, nS1], 'layer2':[nC2, K2, nP2, nS2]})
                # Solutions.append({'im_res_orig':N1, 'im_res_new':N2, 'layer1_old':[C1, Z1, P1, S1], 'layer2_old':[B1, ?, Q1, R1], 'layer1':[C2, Z2, P2, S2], 'layer2':[B2, ?, Q2, R2]})

  return Solutions



In [17]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img


class NetTest(nn.Module):
    def __init__(self, im_res=28, conv1_size=5, conv2_size=3, pool1_size=2, pool2_size=2, num_conv1=10, num_conv2=10, stride_1=1, stride_2=1, padd_1=0, padd_2=0, num_hid=50):
        super(NetTest, self).__init__()
        # import pdb; pdb.set_trace()
        self.conv1 = nn.Conv2d(1, num_conv1, kernel_size=conv1_size, stride=stride_1, padding=padd_1, bias=False)
        self.res_conv1 = int((im_res - conv1_size + 2*padd_1) / stride_1 + 1)
        self.res_pool1 = int((self.res_conv1 - pool1_size + 2*0) / pool1_size + 1)                

        self.conv2 = nn.Conv2d(num_conv1, num_conv2, kernel_size=conv2_size, stride=stride_2, padding=padd_2, bias=False)
        
        self.res_conv2 = int((self.res_pool1 - conv2_size + 2*padd_2) / stride_2 + 1 )       
        self.res_pool2 = int((self.res_conv2 - pool2_size + 2*0) / pool2_size + 1)               
        
        # self.conv2_drop = nn.Dropout2d()

        self.fc1 = nn.Linear((self.res_pool2**2)*num_conv2, num_hid, bias=False)        
        self.fc2 = nn.Linear(num_hid, 10, bias=False)
        self.pool1_size = pool1_size
        self.pool2_size = pool2_size
        self.num_conv2 = num_conv2
        self.im_res = im_res


        
    def forward(self, x):
        # import pdb; pdb.set_trace()

        tmp1 = F.relu(F.max_pool2d(self.conv1(x), self.pool1_size))
        # tmp = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(tmp1)), self.pool2_size))
        tmp = F.relu(F.max_pool2d(self.conv2(tmp1), self.pool2_size))
        # x = F.avg_pool2d(tmp, kernel_size=tmp.shape[-1])
        # x = x[:,:,0,0]

        x = tmp.view(-1, (self.res_pool2**2)*self.num_conv2)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x   ### SUPER IMPORTANT ***************************** works with cross entropy loss loss now

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# class NetTest(nn.Module):
#     def __init__(self, im_res=28, conv1_size=5, conv2_size=5, pool1_size=2, pool2_size=2, num_conv1=10, num_conv2=10, stride_1=1, stride_2=1, padd_1=0, padd_2=0, num_hid=50):
#         super(NetTest, self).__init__()
#         # import pdb; pdb.set_trace()
#         self.conv1 = nn.Conv2d(1, num_conv1, kernel_size=conv1_size, stride=stride_1, padding=padd_1, bias=False)
#         self.res1 = int((im_res - conv1_size + 2*padd_1) / stride_1 + 1)
#         self.conv2 = nn.Conv2d(num_conv1, num_conv2, kernel_size=conv2_size, stride=stride_2, padding=padd_2, bias=False)
#         self.res2_pool = int((self.res1 - pool1_size + 2*0) / pool1_size + 1)                
#         self.res2_conv = int((self.res2_pool - conv2_size + 2*padd_2) / stride_2 + 1 )       
#         self.res2_pool = int((self.res2_conv - pool2_size + 2*0) / pool2_size + 1)               
#         self.conv2_drop = nn.Dropout2d()

#         self.fc1 = nn.Linear((self.res2_pool**2)*num_conv2, num_hid, bias=False)        
#         self.fc2 = nn.Linear(num_hid, 10, bias=False)
#         self.pool1_size = pool1_size
#         self.pool2_size = pool2_size
#         self.num_conv2 = num_conv2
#         self.im_res = im_res

        
#     def forward(self, x):
#         # import pdb; pdb.set_trace()
#         # x = F.avg_pool2d(x, kernel_size=28//self.im_res)
#         # x = F.avg_pool2d(x, kernel_size=2, padding=6, stride=2)
#         # x = cv2.resize(x, (self.im_res, self.im_res)) 

#         # xx = torch.stack((x[0,0],x[0,0], x[0,0]), axis=2)
#         # img_pil = array_to_img(xx.cpu())
#         # img_pil = img_pil.resize((self.im_res,self.im_res))
#         # x = torch.tensor(img_to_array(img_pil)).cuda()
#         # x = x[:,:,0][None,None] #.permute(2,0,1)

#         tmp1 = F.relu(F.max_pool2d(self.conv1(x), self.pool1_size))
#         tmp = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(tmp1)), self.pool2_size))
#         # x = F.avg_pool2d(tmp, kernel_size=tmp.shape[-1])
#         # x = x[:,:,0,0]

#         x = tmp.view(-1, (self.res2_pool**2)*self.num_conv2)
#         x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
#         x = self.fc2(x)
#         return x   ### SUPER IMPORTANT ***************************** works with cross entropy loss loss now

# device = 'cuda'

In [18]:
# c = 6
# p = 3
# s = 2
# assert 20 == (28 - c + 2*p) / s + 1, False
# 19 = 22 + 2*6 
# import cv2


In [19]:
def train_model(model):

  # EPOCHS = 10
  losses = []

  optimizer = optim.Adadelta(model.parameters())

  model.train()
  for epoch in range(EPOCHS):
      for batch_idx, (data, target) in enumerate(train_loader):
          
          data, target = data.to(device), target.to(device)        
          optimizer.zero_grad()
          y_pred = model(data) 

          # import pdb; pdb.set_trace()
          loss = F.cross_entropy(y_pred, target)
          losses.append(loss.cpu().data)

          loss.backward()
          optimizer.step()
          
          # Display
          if batch_idx % 100 == 1:
              print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch+1,
                  EPOCHS,
                  batch_idx * len(data), 
                  len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), 
                  loss.cpu().data), 
                  end='')

  # Eval
  # model.eval()
  # Eval
  model.eval()
  correct = total = 0
  for idx, (images, labels) in enumerate(test_loader):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      _, pre = torch.max(outputs.data, 1)

      total += len(labels)
      correct += (pre == labels).sum()

  accuracy = float(correct) / total
  print('Test Accuracy: %f %%' % accuracy)
  # return acc


  # import pdb; pdb.set_trace()
  # evaluate_x = test_loader.dataset.test_data.type_as(torch.FloatTensor()).to(device)
  # evaluate_y = test_loader.dataset.test_labels.to(device)

  # output = model(evaluate_x[:,None,...])
  # pred = output.data.max(1)[1]
  # d = pred.eq(evaluate_y.data).cpu()
  # accuracy = d.sum().type(dtype=torch.float64)/d.size()[0]
  
  print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.4f}%'.format(
      epoch+1,
      EPOCHS,
      len(train_loader.dataset), 
      len(train_loader.dataset),
      100. * batch_idx / len(train_loader), 
      loss.cpu().data,
      accuracy*100,
      end=''))
  

  return accuracy

In [20]:
# sol_3

In [21]:
def evaluate_orig_model(sol):

  # run the original model for a couple of times
  # sol = sol_3[0]
  sol['orig_solutions'] = []

  im_res = sol['im_res_orig']
  cv1_size = int(sol['layer1_old'][0])
  cv2_size = int(sol['layer2_old'][0])
  no_conv1 = sol['layer1_old'][1]
  no_conv2 = sol['layer2_old'][1]
  st1 = sol['layer1_old'][3]
  st2 = sol['layer2_old'][3]
  pd1 = sol['layer1_old'][2]
  pd2 = sol['layer2_old'][2]

  global train_loader
  global test_loader
  
  train_loader = torch.utils.data.DataLoader(
      datasets.FashionMNIST('../data', train=True, download=True, transform=transforms.Compose([
              transforms.ToTensor(),transforms.Resize((im_res, im_res)),])), batch_size=100, shuffle=True)

  test_loader = torch.utils.data.DataLoader(
      datasets.FashionMNIST('../data', train=False, download=True, transform=transforms.Compose([
              transforms.ToTensor(),transforms.Resize((im_res, im_res)),])), batch_size=100, shuffle=True)

  # for run_no in range(1):
  # try:
    # print(f'----  im_res={im_res_new}, conv1_size={cv1_size}, conv2_size={cv2_size}')
    # import pdb; pdb.set_trace()
  accs = []
  for i in range(NUM_RUNS):
    model = NetTest(im_res=im_res, conv1_size=cv1_size, conv2_size=cv2_size, num_conv1=no_conv1, num_conv2=no_conv2, padd_1=pd1, padd_2=pd2, stride_1=st1, stride_2=st2).to(device)
    if i==0:
      macs, params = profile(model, inputs=(torch.randn(1, 1, im_res, im_res).to(device), ))
    accs.append(train_model(model))
  
  sol['orig_solutions'].append((macs, params, np.mean(accs)))
  # accuracies.append((sol, macs, params, acc))
  # print(accuracies[-1])
  print(sol, '\n')

  # except:
  #   print('\n Model invalid')


  return #sol


In [22]:
def evaluate_new_model(sols):
  # running the solutions; one by one; three each
  # for run_no

  for sol in sols:
        im_res = sol['im_res_new']
        cv1_size = int(sol['layer1'][0])
        cv2_size = int(sol['layer2'][0])
        no_conv1 = sol['layer1'][1]
        no_conv2 = sol['layer2'][1]
        pd1 = sol['layer1'][2]
        pd2 = sol['layer2'][2]
        st1 = sol['layer1'][3]
        st2 = sol['layer2'][3]

        sol['new_solutions'] = []

        global train_loader
        global test_loader

        train_loader = torch.utils.data.DataLoader(
            datasets.FashionMNIST('../data', train=True, download=True, transform=transforms.Compose([
                    transforms.ToTensor(),transforms.Resize((im_res, im_res)),])), batch_size=100, shuffle=True)

        test_loader = torch.utils.data.DataLoader(
            datasets.FashionMNIST('../data', train=False, download=True, transform=transforms.Compose([
                    transforms.ToTensor(),transforms.Resize((im_res, im_res)),])), batch_size=100, shuffle=True)


        # for run_no in range(1):
        try:
          # print(f'----  im_res={im_res_new}, conv1_size={cv1_size}, conv2_size={cv2_size}')
          
          accs = []
          for i in range(NUM_RUNS):
            model = NetTest(im_res=im_res, conv1_size=cv1_size, conv2_size=cv2_size, num_conv1=no_conv1, num_conv2=no_conv2, padd_1=pd1, padd_2=pd2, stride_1=st1, stride_2=st2).to(device)
            if i==0:
              macs, params = profile(model, inputs=(torch.randn(1, 1, im_res, im_res).to(device), ))
            accs.append(train_model(model))
          
          # which_model = 'new_res' if a else 'orig_res'
          # sol[which_model] = (macs, params, acc)
          sol['new_solutions'].append((macs, params, np.mean(accs)))
          # accuracies.append((sol, macs, params, acc))
          # print(accuracies[-1])
          print(sol, '\n')

        except:
          print('\n Model invalid')

  
  return #sols

In [23]:
# np.arange(3,8,2)


from IPython.display import HTML
HTML('''<script>
var code_show_err = false; 
var code_toggle_err = function() {
 var stderrNodes = document.querySelectorAll('[data-mime-type="application/vnd.jupyter.stderr"]')
 var stderr = Array.from(stderrNodes)
 if (code_show_err){
     stderr.forEach(ele => ele.style.display = 'block');
 } else {
     stderr.forEach(ele => ele.style.display = 'none');
 }
 code_show_err = !code_show_err
} 
document.addEventListener('DOMContentLoaded', code_toggle_err);
</script>
To toggle on/off output_stderr, click <a onclick="javascript:code_toggle_err()">here</a>.''')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import random 
all_res = {}


for N1 in [16]:#12,16,20,24]:
  all_solutions = []
  for C1 in np.arange(3,6,2): # filter size layer 1 
    for K1 in np.arange(10,21,5): # num filters layer 1   
      for P1 in np.arange(2): # padding layer 1
        for S1 in np.arange(1,2): # stride layer 1    
          for C2 in np.arange(3,6,2): # filter size layer 2
            for K2 in np.arange(10,21,5): # num filters layer 2   
              for P2 in np.arange(2): # padding layer 2
                for S2 in np.arange(1,2): # stride layer 2   
                    xx = find_res_fixed_flops(N1, C1, K1, P1, S1,   C2, K2, P2, S2, 1)
                    if xx:
                      all_solutions.append(xx)
  
  print(len(all_solutions))
  results = []
  random.shuffle(all_solutions)
  all_solutions.sort(key=len, reverse=True)
  all_solutions = all_solutions[:10]
  for idx, sol_3 in enumerate(all_solutions):
    print(f'\n --------------------- solution no {idx} -------------------- \n')

    if len(sol_3) > 4:
      sol_3 = sol_3[-4:]

    evaluate_orig_model(sol_3[0])
    if not sol_3[0]['orig_solutions']:
      break

    evaluate_new_model(sol_3)
    xx = (sol_3[0]['orig_solutions'][0], [sol['new_solutions'][0][-1] for sol in sol_3 if sol['new_solutions']])

    results.append(xx)

  all_res[N1] = results          


 Train Epoch: 10/10 [100/60000 (0%)]	Loss: 0.858585
 Model invalid

 --------------------- solution no 2 -------------------- 

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.NetTest'>. Treat it as zero Macs and zero Params.
 Train Epoch: 10/10 [50100/60000 (84%)]	Loss: 0.207876Test Accuracy: 0.876900 %
 Train Epoch: 10/10 [60000/60000 (100%)]	Loss: 0.301660	 Test Accuracy: 87.6900%
 Train Epoch: 10/10 [50100/60000 (84%)]	Loss: 0.269761Test Accuracy: 0.874200 %
 Train Epoch: 10/10 [60000/60000 (100%)]	Loss: 0.475185	 Test Accuracy: 87.4200%
 Train Epoch: 10/10 [50100/60000 (84%)]	Loss: 0.321342Test Accuracy: 0.876600 %
 Train Epoch: 10/10 [60000/60000 (100%)]	Loss: 0.526810	 Test Accuracy: 87.6600%
{'im_res_orig': 16, 'im_res_new': 26, 'layer1_old': [3, 20, 1, 1], 'layer2_old': [5, 20, 0, 1], 'layer1': [4, 16, 1, 1], 'layer2': [3, 20, 0, 

In [25]:
sol_3[0]
# all_solutions[0]

{'im_res_new': 28,
 'im_res_orig': 12,
 'layer1': [5, 7, 3, 3],
 'layer1_old': [3, 15, 0, 1],
 'layer2': [5, 20, 0, 1],
 'layer2_old': [5, 20, 0, 1],
 'orig_solutions': []}

In [ ]:
# ([len(x) for x in all_solutions])
# sol_3
# 
# len(all_solutions)



all_res[N1]

In [ ]:
all_solutions[2]

In [ ]:
for k in all_res.keys():
  means = np.mean(np.array([(x[-1],np.max(y)) for x,y in all_res[k]]), axis=0)
  # stds = np.std(np.array([(x[0][-1],np.max(y)) for x,y in all_res[k] ]), axis=0)
  plt.plot(means)

In [ ]:
means

In [ ]:
for k in all_res.keys():
  means = np.mean(np.array([(x[0][-1],y) for x,y in all_res[k] ]), axis=0)
  stds = np.std(np.array([(x[0][-1],y) for x,y in all_res[k] ]), axis=0)
  plt.plot(means)


In [ ]:
sol_3

In [ ]:
# FLOPS
# http://www.bnikolic.co.uk/blog/python/flops/2019/10/01/pytorch-count-flops.html

In [ ]:
aa = {2:'a'}

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
colors = ['blue', 'red', 'green', ]
for idx,i in enumerate([7,14,28]):
  plt.plot(np.log(pars[ress==i]), score[ress==i] , 'o', color = colors[idx])

plt.legend(['res = 7', 'res = 14', 'res = 28'])

In [ ]:
colors = ['blue', 'red', 'green', ]
for idx,i in enumerate([7,14,28]):
  plt.plot(np.log(flops[ress==i]), score[ress==i] , 'o', color = colors[idx])

plt.legend(['res = 7', 'res = 14', 'res = 28'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ls MyDrive/

In [ ]:
cd drive/

In [ ]:
# cd /MyDrive/classification_images/Pairwise
!cd MyDrive/ClassificationImages/Pairwise/

In [ ]:
# from google.colab import files
# # 
# with open('example.txt', 'w') as f:
#   f.write('some content')

# files.download('example.txt')



# with open('MyDrive/ClassificationImages/Pairwise/accs_mnist.npz', 'w') as f:
np.savez('MyDrive/ClassificationImages/Pairwise/accs_fmnist.npz', accs)


In [ ]:
# with open('MyDrive/ClassificationImages/Pairwise/accs_mnist.npz') as f:
dd = np.load('MyDrive/ClassificationImages/Pairwise/accs_mnist.npz')

In [ ]:
# accs = dd['arr_0']

In [ ]:
def test_f(aa):
  aa.append(10)

  return

q = [1]
test_f(q)
print(q)
